In [ ]:
# evaluation.py
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def calculate_metrics(y_true, y_pred, average='weighted'):
  """Calculates common classification metrics."""
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred, average=average, zero_division=0)
  recall = recall_score(y_true, y_pred, average=average, zero_division=0)
  f1 = f1_score(y_true, y_pred, average=average, zero_division=0)
  return accuracy, precision, recall, f1

def plot_confusion_matrix(y_true, y_pred, labels, model_name="Model"):
    """Plots and saves a confusion matrix."""
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'confusion_matrix_{model_name}.png')  # Saving the matrix as an image
    plt.show()

def generate_detailed_report(model, X_test, y_test, model_name="Model"):
    """Generates a detailed report including metrics and confusion matrix."""
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, zero_division=0)
    labels = sorted(list(set(y_test)))
    accuracy, precision, recall, f1 = calculate_metrics(y_test, y_pred)
    print(f"Detailed Report for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Classification Report:\n", report)
    plot_confusion_matrix(y_test, y_pred, labels, model_name)

def plot_feature_importance(model, feature_names, model_name="Model"):
    """Plots feature importances for tree-based models (e.g., Random Forest)."""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        feature_importances = pd.Series(importances, index=feature_names)
        feature_importances = feature_importances.nlargest(10) # Plot top 10
        plt.figure(figsize=(10, 6))
        sns.barplot(x=feature_importances, y=feature_importances.index)
        plt.title(f'Feature Importances - {model_name}')
        plt.xlabel('Importance Score')
        plt.ylabel('Feature')
        plt.savefig(f'feature_importances_{model_name}.png')
        plt.show()
    else:
        print(f"Feature importances not available for {model_name}.")


def main():
  try:
    from feature_engineering import create_tfidf_features, split_data, create_average_word_embeddings, create_word2vec_features
    from sentiment_models import train_naive_bayes, train_logistic_regression, train_random_forest, train_svm
    from data_preprocessing import preprocess_dataframe

    data = pd.read_csv("reviews.csv")
    data.dropna(subset=["text", "sentiment"], inplace=True)
    processed_df = preprocess_dataframe(data, "text")
    texts = processed_df["processed_text"].tolist()
    labels = processed_df["sentiment"].tolist()

    #TF-IDF Features
    tfidf_matrix, tfidf_vectorizer = create_tfidf_features(texts)
    X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = split_data(tfidf_matrix, labels)

     # Average word embeddings
    avg_embeddings = create_average_word_embeddings(texts)
    X_train_avg_emb, X_test_avg_emb, y_train_avg_emb, y_test_avg_emb = split_data(avg_embeddings, labels)

     #Word2Vec Features
    word2vec_embeddings, word2vec_model = create_word2vec_features(texts)
    X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = split_data(word2vec_embeddings, labels)


    #Train Naive Bayes TFIDF
    nb_model_tfidf = train_naive_bayes(X_train_tfidf.toarray(), y_train_tfidf)
    generate_detailed_report(nb_model_tfidf, X_test_tfidf.toarray(), y_test_tfidf, "Naive Bayes TF-IDF")

    #Train Logistic Regression TFIDF
    lr_model_tfidf = train_logistic_regression(X_train_tfidf.toarray(), y_train_tfidf)
    generate_detailed_report(lr_model_tfidf, X_test_tfidf.toarray(), y_test_tfidf, "Logistic Regression TF-IDF")

    #Train Random Forest TFIDF
    rf_model_tfidf = train_random_forest(X_train_tfidf.toarray(), y_train_tfidf)
    generate_detailed_report(rf_model_tfidf, X_test_tfidf.toarray(), y_test_tfidf, "Random Forest TF-IDF")

    #Train SVM TFIDF
    svm_model_tfidf = train_svm(X_train_tfidf.toarray(), y_train_tfidf)
    generate_detailed_report(svm_model_tfidf, X_test_tfidf.toarray(), y_test_tfidf, "SVM TF-IDF")

     #Train Naive Bayes Avg Embedding
    nb_model_avg_emb = train_naive_bayes(X_train_avg_emb, y_train_avg_emb)
    generate_detailed_report(nb_model_avg_emb, X_test_avg_emb, y_test_avg_emb, "Naive Bayes Avg Embeddings")

    #Train Logistic Regression Avg Embedding
    lr_model_avg_emb = train_logistic_regression(X_train_avg_emb, y_train_avg_emb)
    generate_detailed_report(lr_model_avg_emb, X_test_avg_emb, y_test_avg_emb, "Logistic Regression Avg Embeddings")

    #Train Random Forest Avg Embedding
    rf_model_avg_emb = train_random_forest(X_train_avg_emb, y_train_avg_emb)
    generate_detailed_report(rf_model_avg_emb, X_test_avg_emb, y_test_avg_emb, "Random Forest Avg Embeddings")

    #Train SVM Avg Embedding
    svm_model_avg_emb = train_svm(X_train_avg_emb, y_train_avg_emb)
    generate_detailed_report(svm_model_avg_emb, X_test_avg_emb, y_test_avg_emb, "SVM Avg Embeddings")

    #Train Naive Bayes Word2Vec
    nb_model_w2v = train_naive_bayes(X_train_w2v, y_train_w2v)
    generate_detailed_report(nb_model_w2v, X_test_w2v, y_test_w2v, "Naive Bayes Word2Vec")

    #Train Logistic Regression Word2Vec
    lr_model_w2v = train_logistic_regression(X_train_w2v, y_train_w2v)
    generate_detailed_report(lr_model_w2v, X_test_w2v, y_test_w2v, "Logistic Regression Word2Vec")

    #Train Random Forest Word2Vec
    rf_model_w2v = train_random_forest(X_train_w2v, y_train_w2v)
    generate_detailed_report(rf_model_w2v, X_test_w2v, y_test_w2v, "Random Forest Word2Vec")
    plot_feature_importance(rf_model_w2v, tfidf_vectorizer.get_feature_names_out(), "Random Forest Word2Vec")

    #Train SVM Word2Vec
    svm_model_w2v = train_svm(X_train_w2v, y_train_w2v)
    generate_detailed_report(svm_model_w2v, X_test_w2v, y_test_w2v, "SVM Word2Vec")

  except FileNotFoundError:
        print("Error: 'reviews.csv' not found. Please provide the data in a 'reviews.csv' file with 'text' and 'sentiment' columns.")
  except KeyError as e:
        print(f"Error: {e} column not found. Ensure your 'reviews.csv' has 'text' and 'sentiment' columns.")
  except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == '__main__':
  main()